In [1]:
from __future__ import print_function

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [3]:
import numpy as np
import scipy.linalg as sl, scipy.optimize as so
import libstempo as lt, libstempo.toasim as lts, libstempo.plot as ltp
import numdifftools as nd
import matplotlib.pyplot as plt
import os, sys, glob
import tempfile
import pickle
import piccard as pic
import corner
from nutstrajectory import nuts6

%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
def monkey__init__(self, likob, p, hessian, **kwargs):
    """Initialize the whitened likelihood with a maximum and a hessian of
    the parameter space
    """
    self.likob = likob
    self._mu = p.copy()

    # Set the inverse square root
    self.calc_invsqrt(hessian)

    # TODO: Do this through the original __init__ of likelihoodWrapper
    self.initBounds()
    self._cachefunc = None
    self._p = None              # Cached values for coordinates
    self._x = None              # Cached values for coordinates

In [5]:
pic.whitenedLikelihood.__init__ = monkey__init__

In [6]:
if False:
    os.environ['PSR'] = 'J1713+0747'
    os.environ['PSROPT'] = ''
    os.environ['MAXOBS'] = '30000'

In [7]:
psr = os.environ['PSR']
sfx = os.environ['PSROPT'] # '-outno' for no outliers

try:
    maxobs = int(os.environ['MAXOBS'])
except KeyError:
    maxobs = 20000

In [13]:
parfile = glob.glob('working/partim/{0}*.par'.format(psr))[0]
timfile = glob.glob('working/partim/{0}*.tim'.format(psr))[0]
print(parfile,timfile)

working/partim/J1713+0747.ihs.par working/partim/J1713+0747.djn2.tim


In [17]:
h5file = psr + sfx + '.h5'

if not os.path.isfile(h5file):
    # add to a piccard datafile

    t2df = pic.DataFile(h5file)
    t2df.addTempoPulsar(parfile,timfile,maxobs=maxobs)

In [18]:
# create a Piccard likelihood object

likob = pic.hmcLikelihood(h5file) #, sort='jitterext')
modeldict = likob.makeModelDict(nfreqs=20, incRedNoise=True, noiseModel='powerlaw',
                                incOutliers=('outno' not in sfx), \
                                noisePrior="flatlog", varyEfac=True, separateEfacs=True, \
                                incEquad=True, separateEquads=True, \
                                ndmfreqs=0, incDM=False, dmModel='dmpowerlaw', \
                                incCEquad=True, separateCEquads=True, \
                                incBWM=False, incGWB=False, gwbModel='powerlaw', \
                                incPsrBWM=False, signPsrBWM=-0.5, varyBWMSign=False, nPsrBWM=1, \
                                likfunc='mark15' if ('outno' not in sfx) else 'mark14',
                                expandJitter=True, priorDraws=False)
likob.initModel(modeldict, fromFile=False, verbose=False)

In [70]:
reslice = np.arange(0, likob.dimensions)

parnames = []
for pd in likob.pardes:
    ii = pd['index']
    if ii >= 0 and ii in reslice:
        print(ii,pd['id'])
        parnames.append(pd['id'])

nhyperpars = parnames.index('Offset')

0 efacJ1713+0747-Rcvr_800_GUPPI
1 efacJ1713+0747-Rcvr1_2_GASP
2 efacJ1713+0747-L-wide_ASP
3 efacJ1713+0747-Rcvr1_2_GUPPI
4 efacJ1713+0747-S-wide_ASP
5 efacJ1713+0747-Rcvr_800_GASP
6 efacJ1713+0747-L-wide_PUPPI
7 efacJ1713+0747-S-wide_PUPPI
8 equadJ1713+0747-Rcvr_800_GUPPI
9 equadJ1713+0747-Rcvr1_2_GASP
10 equadJ1713+0747-L-wide_ASP
11 equadJ1713+0747-Rcvr1_2_GUPPI
12 equadJ1713+0747-S-wide_ASP
13 equadJ1713+0747-Rcvr_800_GASP
14 equadJ1713+0747-L-wide_PUPPI
15 equadJ1713+0747-S-wide_PUPPI
16 jitterJ1713+0747-Rcvr_800_GUPPI
17 jitterJ1713+0747-Rcvr1_2_GASP
18 jitterJ1713+0747-L-wide_ASP
19 jitterJ1713+0747-Rcvr1_2_GUPPI
20 jitterJ1713+0747-S-wide_ASP
21 jitterJ1713+0747-Rcvr_800_GASP
22 jitterJ1713+0747-L-wide_PUPPI
23 jitterJ1713+0747-S-wide_PUPPI
24 RN-Amplitude
25 RN-spectral-index
26 outlier_prob_J1713+0747
27 Offset
28 ELONG
29 ELAT
30 F0
31 F1
32 PMELONG
33 PMELAT
34 PX
35 PB
36 T0
37 A1
38 OM
39 ECC
40 PBDOT
41 XDOT
42 OMDOT
43 M2
44 FD1
45 FD2
46 FD3
47 FD4
48 KOM
49 KIN
50 DMX_

In [71]:
# Optimize the posterior
def func(x):
    ll, _ = likob.logposterior_grad(x)
    
    return -np.inf if np.isnan(ll) else ll

def jac(x):
    _, j = likob.logposterior_grad(x)
    return j

def hess(x):
    return likob.hessian(x)

In [72]:
%timeit func(likob.pstart)

1 loop, best of 3: 3.43 s per loop


In [73]:
func(likob.pstart)

356261.17328137311

In [27]:
# cd runs

/Volumes/NANOGrav/outliers


In [74]:
endpfile = psr + sfx + '-endp.pickle'

In [76]:
%%time
if not os.path.isfile(endpfile):
    # Use L-BFGS-B for larger problems
    endp = likob.pstart
    
    for iter in range(3):
        res = so.minimize(lambda x: -func(x),
                          endp,
                          jac=lambda x: -jac(x),
                          hess=None,
                          method='L-BFGS-B', options={'disp': True})

        endp = res['x']
    
    pickle.dump(endp,open(endpfile,'wb'))
else:
    endp = pickle.load(open(endpfile,'rb'))

CPU times: user 827 µs, sys: 1.46 ms, total: 2.29 ms
Wall time: 7.67 ms


In [77]:
func(endp)

363097.05962849449

In [78]:
hessfile = psr + sfx + '-fullhessian.pickle'

In [79]:
%%time
if not os.path.isfile(hessfile):
    reslice = np.arange(0,nhyperpars)

    def partfunc(x):
        p = np.copy(endp)
        p[reslice] = x
        return likob.logposterior_grad(p)[0]

    ndhessdiag = nd.Hessdiag(func)
    ndparthess = nd.Hessian(partfunc)
    
    # Create a good-enough approximation for the Hessian
    nhdiag = ndhessdiag(endp)
    nhpart = ndparthess(endp[reslice])
    fullhessian = np.diag(nhdiag)
    fullhessian[:nhyperpars,:nhyperpars] = nhpart
    
    pickle.dump(fullhessian,open(hessfile,'wb'))
else:
    fullhessian = pickle.load(open(hessfile,'rb'))

CPU times: user 138 ms, sys: 75.5 ms, total: 214 ms
Wall time: 1.1 s


In [80]:
# Create a whitened likelihood
likob.pstart = endp
wl = pic.whitenedLikelihood(likob,endp,-fullhessian)
wlps = wl.pstart

In [81]:
likob.logposterior(endp)

363097.05962849449

In [82]:
wl.likob.logposterior(wl.backward(wlps))

363097.05962849205

In [83]:
chaindir = 'chains_' + psr + sfx

In [37]:
!mkdir -p {chaindir}

In [84]:
%%time
if not os.path.isfile(chaindir + '/samples.txt'):
    # Run NUTS for 20000 samples, with a burn-in of 1000 samples (target acceptance = 0.6)
    samples, lnprob, epsilon = nuts6(wl.logposterior_grad,20000,1000,
                                     wlps,0.6,
                                     verbose=True,
                                     outFile=chaindir + '/samples.txt')

CPU times: user 173 µs, sys: 186 µs, total: 359 µs
Wall time: 179 µs


In [85]:
parsfile = psr + sfx + '-pars.npy'

In [86]:
%%time
if not os.path.isfile(parsfile):
    samples = np.loadtxt(chaindir + '/samples.txt')
    pars = wl.fullbackward(samples[:,:-2])
    np.save(parsfile,pars)
else:
    pars = np.load(parsfile)

CPU times: user 891 µs, sys: 109 ms, total: 110 ms
Wall time: 3.1 s


In [87]:
if not os.path.isfile(psr + sfx + '-corner.pdf'):
    corner.corner(pars[:,:nhyperpars],labels=parnames[:nhyperpars],show_titles=True);
    plt.savefig(psr + sfx + '-corner.pdf')

In [88]:
pars.shape

(14592, 1153)

In [89]:
likob.ptapsrs[0].P0

0.00457013652727228

In [90]:
def poutlier(p,likob):
    """Invoked on a sample parameter set and the appropriate likelihood,
    returns the outlier probability (a vector over the TOAs) and
    the individual sqrt(chisq) values"""
    
    # invoke the likelihood
    _ = likob.mark15loglikelihood(p)

    # get the piccard pulsar object
    psr = likob.ptapsrs[0]

    r = psr.detresiduals
    N = psr.Nvec

    Pb = psr.outlier_prob # a priori outlier probability for this sample
    P0 = psr.P0           # width of outlier range
    
    PA = 1.0 - Pb
    PB = Pb
    
    PtA = np.exp(-0.5*r**2/N) / np.sqrt(2*np.pi*N)
    PtB = 1.0/P0
    
    num = PtB * PB
    den = PtB * PB + PtA * PA
    
    return num/den, r/np.sqrt(N)

In [91]:
pobsfile = psr + sfx + '-pobs.npy'

In [92]:
%%time
if 'outno' not in sfx:
    if not os.path.isfile(pobsfile):
        nsamples = len(pars)
        nobs = len(likob.ptapsrs[0].Nvec)

        # basic likelihood
        lo = wl.likob.likob.likob

        outps = np.zeros((nsamples,nobs),'d')
        sigma = np.zeros((nsamples,nobs),'d')

        for i,p in enumerate(pars):
            outps[i,:], sigma[i,:] = poutlier(p,lo)

        out = np.zeros((nsamples,nobs,2),'d')
        out[:,:,0], out[:,:,1] = outps, sigma    
        np.save(pobsfile,out)
    else:
        out = np.load(pobsfile)
        outps, sigma = out[:,:,0], out[:,:,1]

CPU times: user 8.54 ms, sys: 5.46 s, total: 5.47 s
Wall time: 2min 27s


In [93]:
if 'outno' not in sfx:
    avgps = np.mean(outps,axis=0)
    medps = np.median(outps,axis=0)

In [95]:
residualplot = psr + sfx + '-residuals.pdf'

if not os.path.isfile(residualplot):
    outliers = medps > 0.1
    nout = np.sum(outliers)
    nbig = nout
    
    print("Big: {}".format(nbig))
    
    if nout == 0:
        outliers = medps > 5e-4
        nout = np.sum(outliers)
    
    print("Plotted: {}".format(nout))

    plt.figure(figsize=(9,4))

    psrobj = likob.ptapsrs[0]

    # convert toas to mjds
    toas = psrobj.toas/pic.pic_spd + pic.pic_T0

    # red noise at the starting fit point
    _ = likob.loglikelihood(endp)
    rednoise = psrobj.residuals - psrobj.detresiduals

    # plot tim-file residuals (I think)
    plt.errorbar(toas,psrobj.residuals,yerr=psrobj.toaerrs,fmt='.',alpha=0.3)

    # red noise
    # plt.plot(toas,rednoise,'r-')

    # possible outliers
    plt.errorbar(toas[outliers],psrobj.residuals[outliers],yerr=psrobj.toaerrs[outliers],fmt='rx')

    plt.savefig(residualplot)

In [120]:
report = psr + sfx + '-outliers.txt'

if 'outno' not in sfx and not os.path.isfile(report):
    t2 = lt.tempopulsar(parfile,timfile,maxobs=maxobs)

    outliers = medps > 0.1
    nout = np.sum(outliers)

    isort = wl.ptapsrs[0].isort
    
    with open(report,'w') as outfile:
        print('# {}'.format(psr),file=outfile)
        print('# using parfile={}, timfile={}'.format(parfile,timfile),file=outfile)
        print('# {} strong (median p > 0.1) outlier candidates'.format(nout),file=outfile)
        
        largest = (-medps).argsort()[:nout]

        for i,k in enumerate(largest):
            j = isort[k]
            print('{filename} {freq:.6f} {toa} -f {f} -chan {chan} -subint {subint} -snr {snr} -pout {pout:.2e}'.format(
                      filename=t2.filename()[j],
                      freq=t2.freqs[j],
                      toa=repr(t2.stoas[j]),
                      f=t2.flagvals('f')[j],
                      chan=t2.flagvals('chan')[j],
                      subint=t2.flagvals('subint')[j],
                      snr=t2.flagvals('snr')[j],
                      pout=medps[k]),
                  file=outfile)

    del t2

In [38]:
largestplot = psr + sfx + '-largest.pdf'

if 'outno' not in sfx and not os.path.isfile(largestplot):
    outliers = medps > 0.1
    nout = np.sum(outliers)
    
    largest = (-medps).argsort()[:min(nout,10)]

    ncol = len(largest)
    plt.figure(figsize=(9,3*ncol))

    for i,j in enumerate(largest):
        plt.subplot(ncol,2,2*i+1)
        plt.hist(outps[:,j],bins=40,normed=True);
        plt.xlabel('p_out')
        plt.title('TOA # {}, t = {}'.format(j,toas[j]))

        plt.subplot(ncol,2,2*i+2)
        plt.plot(np.abs(sigma[:,j]),outps[:,j],'.',alpha=0.1)
        plt.xlabel('res/sigma'); plt.ylabel('p_out')
        plt.title('median p = {:.2e}'.format(medps[j]))

    plt.tight_layout()

    plt.savefig(largestplot)

In [39]:
import scipy.interpolate as interp
import scipy.ndimage.filters as filter

def getMax(samples,weights=None,range=None,bins=50):
   if range is None:
       hist, xedges = np.histogram(samples, bins, normed=True, weights=weights)
   else:
       hist, xedges = np.histogram(samples, bins, normed=True, range=range,\
                                  weights=weights)

   xedges = np.delete(xedges, -1) + 0.5*(xedges[1] - xedges[0])

   # gaussian smoothing
   hist = filter.gaussian_filter(hist, sigma=0.75)

   # interpolation
   f = interp.interp1d(xedges, hist, kind='cubic')
   xedges = np.linspace(xedges.min(), xedges.max(), 10000)
   hist = f(xedges)

   return xedges[np.argmax(hist)]

In [40]:
import re

noisefile = psr + sfx + '-noise.txt'

if not os.path.isfile(noisefile):
    with open(noisefile,'w') as outfile:
        for i in range(nhyperpars):
            parname = re.sub(re.sub('\+','\\+',psr),'',parnames[i])
            if parname[-1] == '_':
                parname = parname[:-1]
            print(parname,getMax(pars[:,i]),file=outfile)